In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import re 
import requests

In [8]:
boptions = webdriver.ChromeOptions()
boptions.add_argument("----headless")
driver = webdriver.Chrome(options=boptions)
driver.get('https://ria.ru/economy')
data = []
last_i = -1

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    containers = soup.find_all("div", attrs={"class": "list-item", "data-type": "article"})

    for i, container in enumerate(containers[last_i:]):

        if i < last_i:
            continue
        
        #header = container.find('a', attrs= {'class': re.compile('^list-item__title\D*$')})
        header = container.find('a', attrs= {'class': 'list-item__title'})
        if header is None: header = container.find('a', attrs= {'class': 'list-item__title color-font-hover-only'})
        header_text = header.text
        href = header['href']

        # open the page
        page = requests.get(href)
        soup_inner = BeautifulSoup(page.text, 'html.parser')
        dttime_el = soup_inner.find('div', attrs={'class': 'article__info-date'}).find('a')
        dttime = dttime_el.text if dttime_el is not None else "None"
        #author = "NOT FOUND"
        brief_el = soup_inner.find('h1', attrs={'class': 'article__second-title'})
        brief = "None" if brief_el is None else brief_el.text
        tags_element = soup_inner.find('div', attrs={'class': 'article__tags'}).find_all('a')
        tags = ",".join([el.text for el in tags_element])


        #//*[@id="content"]/div/div[1]/div/div[3]

        print(i, header_text, dttime, href, brief, tags)
        data.append((header_text, dttime, href, brief, tags)) 

        last_i = i

        #data.append((header, timedata, ref))
        #print(f"Found header: {header}, date: {timedata}, ref: {ref}")

    #//*[@id="content"]/div/div[1]/div/div[3]
    bottom = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/div[3]')
    location = bottom.location
    size = bottom.size
    action = ActionChains(driver) 
    action.move_to_element(bottom).perform()
    driver.execute_script("arguments[0].click();", bottom)
    time.sleep(1)


0 Россия нарастила поставки кукурузы в Китай в 90 раз в апреле  11:42 20.05.2025 https://ria.ru/20250520/rossija-2018017812.html Россия увеличила поставки кукурузы в Китай в 90 раз до 3 млн долларов в апреле Экономика,Китай,Россия,Австралия
0 Украина не способна сама себя обслуживать, заявил Медведев  18:08 20.05.2025 https://ria.ru/20250520/ukraina-2018109065.html Медведев: Украина не способна сама себя обслуживать во всех смыслах Экономика,Украина,Россия,Санкт-Петербург,Дмитрий Медведев,Петербургский юридический форум — 2025,В мире
1 Медведев предположил, каким будет документ, который подпишут с Украиной  17:34 20.05.2025 https://ria.ru/20250520/ukraina-2018100308.html Медведев: предстоит большая работа над документом по урегулированию на Украине Специальная военная операция на Украине,Экономика,Украина,Россия,Санкт-Петербург,Дмитрий Медведев,Петербургский юридический форум — 2025
2 Медведев заявил о неприятии финансирования Киева Всемирным банком  17:17 20.05.2025 https://ria.ru/202

KeyboardInterrupt: 

In [5]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link', 'brief', 'tags'])
df = df.drop_duplicates()
df.tail()

,header,date,link,brief,tags
7801,"""Газпром"" подает газ через Украину на ГИС ""Суд...",10:01 11.10.2024,https://ria.ru/20241011/gaz-1977527463.html,"""Газпром"" подает газ через Украину в объеме 42...","Экономика,Курская область,Европа,Украина,Серге..."
7802,СМИ рассказали о возможных изменениях в сфере ...,09:46 11.10.2024,https://ria.ru/20241011/rbk-1977523205.html,РБК: власти предложили запретить списание сред...,"Экономика,Министерство промышленности и торгов..."
7803,Глава Евросовета призвал Китай изменить поведе...,08:33 11.10.2024,https://ria.ru/20241011/kitay-1977514038.html,Глава Евросовета призвал Китай изменить поведе...,"Экономика,Китай,Брюссель,Пекин,Шарль Мишель,Ев..."
7804,Россия включила на полную мощность свое самое ...,08:00 11.10.2024,https://ria.ru/20241011/oruzhie-1977493498.html,None,"Экономика,Россия,Австралия,США,Владимир Путин,..."
7805,"""Надежный способ"". Москва намекнула, к чему го...",08:00 11.10.2024,https://ria.ru/20241011/zoloto-1977451201.html,Финансист Щербаченко: смысл инвестиций в золот...,"Экономика,Россия,США,Центральный Банк РФ (ЦБ РФ)"


In [6]:
df.to_csv("../data/ria_20052025_11102024.csv", index=False)